### Extraction des données du  RAD exportées dans BQ

In [208]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np,io,codecs,json
from random import *
from datetime import datetime
import matplotlib.colors as mcolors
from ipywidgets import *
from google.cloud import bigquery
from google.oauth2 import service_account
cleSec=widgets.Text( description='Entrer la clé : ')

In [221]:
print('Pour utiliser ce service, il faut disposer d\'un compte sur Google Cloud Platform (GCP) et créer un "service_account". ')
print('GCP permet de créer un fichier (.json) qui contient la clé de ce compte. ')
print('Importer ci-dessous ("upload") le fichier de la clé du "service_account". ')
def on_upload_change(change):
    global client
    upl_dc=codecs.decode((list(upl.value.values())[0]['content']))
    credentials=service_account.Credentials.from_service_account_info(json.loads(upl_dc))
    client = bigquery.Client(credentials= credentials,project='dirif25')
upl=FileUpload()
display(upl)
upl.observe(on_upload_change, names='_counter')

Pour utiliser ce service, il faut disposer d'un compte sur Google Cloud Platform (GCP) et créer un "service_account". 
GCP permet de créer un fichier (.json) qui contient la clé de ce compte. 
Importer ci-dessous ("upload") le fichier de la clé du "service_account". 


FileUpload(value={}, description='Upload')

In [217]:
quer="""
    Select distinct Stat,Axe from `dirif25.6mn.lstSt`
    """
lstSt=client.query(quer).to_dataframe()
lstAx=list(lstSt['Axe'].unique())
dic={}
for x in lstAx:
    dic[x]=list(lstSt[lstSt['Axe']==x]['Stat'].values)

In [130]:
print('Choisir, dans un premier temps, l\'axe du point de mesure, dans la liste déroulante.\nPuis, sélectionner le point de mesure.')
axeW = Dropdown(options = dic.keys())
statW = Dropdown(options = dic[axeW.value])
@interact(Axe = axeW, PointMesure = statW)
def print_city(Axe, PointMesure):
   statW.options = dic[Axe] 

Choisir, dans un premier temps, l'axe du point de mesure, dans la liste déroulante.
Puis, sélectionner le point de mesure.


interactive(children=(Dropdown(description='Axe', options=('A1-W', 'A1-Y', 'A3-W', 'A3-Y', 'A4-W', 'A4-Y', 'A6…

In [138]:
statW.value

'A13-W/03+0400/P'

In [124]:
dtDeb=widgets.DatePicker(description='Début', value=datetime(2019,1,3))
dtFin=widgets.DatePicker(description='Fin', value=datetime(2019,10,31))
print('Renseigner ci-dessous la période pour laquelle faire une extraction')
VBox([dtDeb,dtFin])

Renseigner la période pour laquelle faire une extraction


In [135]:
def qrSt(stV,dtDeb,dtFin):
    d1=str(dtDeb.value.timetuple().tm_yday)
    d2=str(dtFin.value.timetuple().tm_yday)   
    q1 = """
    SELECT P6mn,Db,jr FROM (
        SELECT  Stat,P6mn,Db,EXTRACT(year FROM dt) AS an,EXTRACT(dayofyear FROM dt) AS jr FROM
          `dirif25.6mn.bsDbt1` WHERE Stat=\""""
    q2="\"  ) WHERE  an=2019 and (jr+1)>"
    q3=" and (jr-1)<"
    q4=" order by jr,P6mn"
    query=q1+stV+q2+d1+q3+d2+q4
    res=client.query(query).to_dataframe().astype(int)
    res['date']=pd.to_datetime(2019000+res['jr'],format='%Y%j'   )
    return res[['date','P6mn','Db']]



In [137]:
resultat=qrSt(statW.value,dtDeb,dtFin)
resultat.to_csv('resultat.csv')
resultat

,date,P6mn,Db
0,2019-01-03,1,188
1,2019-01-03,2,159
2,2019-01-03,3,185
3,2019-01-03,4,181
4,2019-01-03,5,171
...,...,...,...
69440,2019-10-31,236,315
69441,2019-10-31,237,341
69442,2019-10-31,238,306
69443,2019-10-31,239,324
